In [2]:
# necessary Modules

import random
from hashlib import sha256, sha3_256

In [3]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    keys_list = []
    for _ in range(n):
        x_temp, y_temp = generate_keys(g, p)
        keys_list.append([x_temp, y_temp])
    return keys_list


def concat_and_shuffle_total_keys(keys_list, x, y):
    keys_list.append([x, y])
    random.shuffle(keys_list)
    return keys_list


def get_real_key_position(keys_list, y):
    public_key_list = [key[1] for key in keys_list]
    return public_key_list.index(y)


def get_Y(keys_list):
    public_key_list = [key[1] for key in keys_list]
    Y = ''.join([str(i) for i in public_key_list])
    return Y


def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R


def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    e_list = [None for _ in range(len(keys_list))]
    sig_list = [None for _ in range(len(keys_list))]
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    y_wave = pow(h, x_real, p)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    e_list[idx] = e_initial
    while keys_list[idx][1] != y_real:
        e_last_one = e_list[idx]
        _, random_sig = generate_keys(g, p)
        sig_list[idx] = random_sig
        temp_y = keys_list[idx][1]
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        idx = (idx + 1) % len(total_keys_list)
        e_list[idx] = temp_e
        
    # q is the order for p, see paper page 6, part 4, the first sentence. 
    q = p - 1
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
        'e_list': e_list
    }
    return total_res
        
    
    

# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
random_keys_count = 10

random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
print(total_keys_list)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)



[[712248996, 664603032], [131056189, 686040541], [218352875, 69406070], [384999500, 21571338], [634299836, 418696564], [134168262, 531033154], [353470094, 922695381], [676020449, 135166152], [414283557, 679990306], [37335846, 926592357], [96374602, 613751589]]


{'e1': 47111303927130558371729202037086723609932139677933603935963851617944420835686,
 'sig_list': [881370404,
  585769225,
  882449527,
  298131238,
  136651118,
  371577616,
  111491706,
  473721463,
  899557843,
  759555295,
  850435196],
 'public_key_list': [664603032,
  686040541,
  69406070,
  21571338,
  418696564,
  531033154,
  922695381,
  135166152,
  679990306,
  926592357,
  613751589],
 'tag': 185721214,
 'msg': 'Hello, Ring Signature',
 'e_list': [47111303927130558371729202037086723609932139677933603935963851617944420835686,
  98225134333774521677269740447249684453883327501947092157798963006837519446155,
  71163050164265827047854608145740619072694770971401609490095885158234486116155,
  77170230064583136480455732443551177704183068917384738918516557632547881125812,
  47581385473974822091619754424202616680142119156390870524548256151342697072179,
  113341114984676806171576405939798474921420151149376647905692516415806357467536,
  7791099448759268888546402551064680549599692024

In [4]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 47111303927130558371729202037086723609932139677933603935963851617944420835686
temp_e_V: 98225134333774521677269740447249684453883327501947092157798963006837519446155
temp_e_V: 71163050164265827047854608145740619072694770971401609490095885158234486116155
temp_e_V: 77170230064583136480455732443551177704183068917384738918516557632547881125812
temp_e_V: 47581385473974822091619754424202616680142119156390870524548256151342697072179
temp_e_V: 113341114984676806171576405939798474921420151149376647905692516415806357467536
temp_e_V: 77910994487592688885464025510646805495996920242336966218501569556499244165209
temp_e_V: 92110003357510575642963321687827121771941207226416349213804344240693680159921
temp_e_V: 50622445171916285881972943432983314467732691863599672835220915150151542681638
temp_e_V: 18620763527331296864989265593002966777422532423587311852815802832850237067733
temp_e_V: 26392061964920997030007175765747467452223723052256421803717813530463628570472
4711130392713055837172920203708

In [7]:
msg_1 = 'new test'

total_res_1 = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
print(display(total_res_1))
print(total_res_1['tag'] == total_res['tag'])

{'e1': 20109497603014776828441311798665477380921256639517370245930503994392475509595,
 'sig_list': [856977903,
  996976730,
  446750408,
  188589917,
  177738267,
  246052167,
  862983367,
  102910180,
  769286149,
  945287543,
  395692106],
 'public_key_list': [664603032,
  686040541,
  69406070,
  21571338,
  418696564,
  531033154,
  922695381,
  135166152,
  679990306,
  926592357,
  613751589],
 'tag': 185721214,
 'msg': 'new test',
 'e_list': [20109497603014776828441311798665477380921256639517370245930503994392475509595,
  73069181243734537663811298891026231875451288413815807220204329927607846880841,
  55495077272290377681204822956889342230163716453990948479896077977586784588926,
  40073531089114829249512406285222235707320077231903384046022013931334456337633,
  79853481054680763236059260156534550430184267048033851096834398303278513458185,
  79279031873795290251898406725123544113386751188993493937347957060668887375074,
  599457209457323040973005610970072646852959338337007085882196

None
True


In [6]:
def is_primitive_root(p, g):
    n = p - 1
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, (p-1) // factor, p) == 1:
            return False
    return True

p = 1000000007 
g = 5 

result = is_primitive_root(p, g)
print(f"g = {g} is a primitive root of p = {p}: {result}")


g = 5 is a primitive root of p = 1000000007: True
